In [1]:
# importing dependencies here
import numpy as np
import pandas as pd
import os

# visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# feature engineering
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download("stopwords")

# sentiment scoring
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# pos tagging
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# accuracy scores
from sklearn.metrics import (
    f1_score,
    accuracy_score,
)

# performance check
import time

# sparse to dense
from sklearn.base import TransformerMixin


class DenseTransformer(TransformerMixin):
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()


# importing model
from joblib import load



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path_to_csv = ("df_holdout.csv")
df = pd.read_csv(path_to_csv)

In [3]:
df.head(2)

,type,posts
0,INFJ,'I have never seen so many poorly used memes.....
1,INFJ,'Wow! You are obviously her muse... Be flatter...


In [4]:
def categorize_types(personality_data):
    personality_data["is_Extrovert"] = personality_data["type"].apply(
        lambda x: 1 if x[0] == "E" else 0
    )
    personality_data["is_Sensing"] = personality_data["type"].apply(
        lambda x: 1 if x[1] == "S" else 0
    )
    personality_data["is_Thinking"] = personality_data["type"].apply(
        lambda x: 1 if x[2] == "T" else 0
    )
    personality_data["is_Judging"] = personality_data["type"].apply(
        lambda x: 1 if x[3] == "J" else 0
    )

    # rearranging the dataframe columns
    personality_data = personality_data[
        ["type", "is_Extrovert", "is_Sensing", "is_Thinking", "is_Judging", "posts"]
    ]

"""
def clean_posts(personality_data):
    # converting posts into lower case
    personality_data["clean_posts"] = personality_data["posts"].str.lower()

    # replacing ||| with space
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"\|\|\|", " "
    )

    # replacing urls with domain name
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"https?:\/\/(www)?.?([A-Za-z_0-9-]+)([\S])*",
        ""
    )

    # dropping emails
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"\S+@\S+", ""
    )

    # dropping punctuations
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"[^a-z\s]", " "
    )

    # dropping MBTIs mentioned in the posts
    mbti = personality_data["type"].unique()
    for type_word in mbti:
        personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
            type_word.lower(), ""
        )
        
    # removing words that are 1 to 2 characters long    
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"\b\w{1,2}\b", ""
    )

    # lemmitizing
    lemmatizer = WordNetLemmatizer()

    personality_data["clean_posts"] = personality_data["clean_posts"].apply(
        lambda x: " ".join(
            [
                lemmatizer.lemmatize(word)
                for word in x.split(" ")
                if word not in stopwords.words("english")
            ]
        )
    )

    # tag_posts will be a list of 50 lists.
    # replacing urls with domain name
    personality_data["tag_posts"] = personality_data["posts"].str.replace(
        r"https?:\/\/(www)?.?([A-Za-z_0-9-]+)([\S])*",
        lambda match: match.group(2),
    )
    # replacing ||| with space
    personality_data["tag_posts"] = [
        post for post in personality_data["tag_posts"].str.split("\|\|\|")
    ]
"""

def clean_posts(personality_data):
    # converting posts into lower case
    personality_data["clean_posts"] = personality_data["posts"].str.lower()

    # replacing ||| with space
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"\|\|\|", " "
    )

    # replacing urls with domain name
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"https?:\/\/(www)?.?([A-Za-z_0-9-]+)([\S])*",
        lambda match: match.group(2) if match.group(2) is not None else "",
        regex=True
    )

    # dropping emails
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"\S+@\S+", ""
    )

    # dropping punctuations
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"[^a-z\s]", " "
    )

    # dropping MBTIs mentioned in the posts
    mbti = personality_data["type"].unique()
    for type_word in mbti:
        personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
            type_word.lower(), ""
        )
        
    # removing words that are 1 to 2 characters long    
    personality_data["clean_posts"] = personality_data["clean_posts"].str.replace(
        r"\b\w{1,2}\b", ""
    )

    # lemmitizing
    lemmatizer = WordNetLemmatizer()

    personality_data["clean_posts"] = personality_data["clean_posts"].apply(
        lambda x: " ".join(
            [
                lemmatizer.lemmatize(word)
                for word in x.split(" ")
                if word not in stopwords.words("english")
            ]
        )
    )

    # tag_posts will be a list of 50 lists.
    # replacing urls with domain name
    personality_data["tag_posts"] = personality_data["posts"].str.replace(
        r"https?:\/\/(www)?.?([A-Za-z_0-9-]+)([\S])*",
        lambda match: match.group(2) if match.group(2) is not None else "",
        regex=True
    )
    # replacing ||| with space
    personality_data["tag_posts"] = [
        post for post in personality_data["tag_posts"].str.split("\|\|\|")
    ]


def sentiment_score(personality_data):
    analyzer = SentimentIntensityAnalyzer()

    nlp_sentiment_score = []

    for post in personality_data["clean_posts"]:
        score = analyzer.polarity_scores(post)["compound"]
        nlp_sentiment_score.append(score)

    personality_data["compound_sentiment"] = nlp_sentiment_score


def pos_tagging(personality_data):
    personality_data["tagged_words"] = personality_data["tag_posts"].apply(
        lambda x: [nltk.pos_tag(word_tokenize(line)) for line in x]
    )

    # grouping pos tags based on stanford list
    tags_dict = {
        "ADJ": ["JJ", "JJR", "JJS"],
        "ADP": ["EX", "TO"],
        "ADV": ["RB", "RBR", "RBS", "WRB"],
        "CONJ": ["CC", "IN"],
        "DET": ["DT", "PDT", "WDT"],
        "NOUN": ["NN", "NNS", "NNP", "NNPS"],
        "NUM": ["CD"],
        "PRT": ["RP"],
        "PRON": ["PRP", "PRP$", "WP", "WP$"],
        "VERB": ["MD", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"],
        ".": ["#", "$", "''", "(", ")", ",", ".", ":"],
        "X": ["FW", "LS", "UH"],
    }

    def stanford_tag(x, tag):
        tags_list = [len([y for y in line if y[1] in tags_dict[col]]) for line in x]
        return tags_list

    for col in tags_dict.keys():
        personality_data[col + "_avg"] = personality_data["tagged_words"].apply(
            lambda x: np.mean(stanford_tag(x, col))
        )


def get_counts(personality_data):
    def unique_words(s):
        unique = set(s.split(" "))
        return len(unique)/50

    def emojis(post):
        emoji_count = 0
        words = post.split()
        for e in words:
            if "http" not in e:
                if e.count(":") == 2:
                    emoji_count += 1
        return emoji_count/50

    def colons(post):
        colon_count = 0
        words = post.split()
        for e in words:
            if "http" not in e:
                colon_count += e.count(":")
        return colon_count/50

    personality_data["qm"] = personality_data["posts"].apply(lambda s: s.count("?")/50)
    personality_data["em"] = personality_data["posts"].apply(lambda s: s.count("!")/50)
    personality_data["colons"] = personality_data["posts"].apply(colons)
    personality_data["emojis"] = personality_data["posts"].apply(emojis)

    personality_data["word_count"] = personality_data["posts"].apply(
        lambda s: (s.count(" ") + 1)/50
    )
    personality_data["unique_words"] = personality_data["posts"].apply(unique_words)

    personality_data["upper"] = personality_data["posts"].apply(
        lambda x: len([x for x in x.split() if x.isupper()])/50
    )
    personality_data["link_count"] = personality_data["posts"].apply(
        lambda s: s.count("http")/50
    )
    ellipses_count = [
        len(re.findall(r"\.\.\.\ ", posts))/50 for posts in personality_data["posts"]
    ]
    personality_data["ellipses"] = ellipses_count
    personality_data["img_count"] = [
        len(re.findall(r"(\.jpg)|(\.jpeg)|(\.gif)|(\.png)", post))/50
        for post in personality_data["posts"]
    ]


In [5]:
def prep_data(personality_data):

    t = time.time()

    categorize_types(personality_data)

    clean_posts(personality_data)

    sentiment_score(personality_data)

    pos_tagging(personality_data)

    get_counts(personality_data)

    features = personality_data[
        [
            "clean_posts",
            "compound_sentiment",
            "ADJ_avg",
            "ADP_avg",
            "ADV_avg",
            "CONJ_avg",
            "DET_avg",
            "NOUN_avg",
            "NUM_avg",
            "PRT_avg",
            "PRON_avg",
            "VERB_avg",
            "qm",
            "em",
            "colons",
            "emojis",
            "word_count",
            "unique_words",
            "upper",
            "link_count",
            "ellipses",
            "img_count",
        ]
    ]

    X = features
    y = personality_data.iloc[:, 2:6]

    print(f"Total Preprocessing Time: {time.time()-t} seconds\n")

    return X, y

In [6]:
def combine_classes(y_pred1, y_pred2, y_pred3, y_pred4):
    
    combined = []
    for i in range(len(y_pred1)):
        combined.append(
            str(y_pred1[i]) + str(y_pred2[i]) + str(y_pred3[i]) + str(y_pred4[i])
        )
    
    result = trace_back(combined)
    return result
    

def trace_back(combined):
        
    type_list = [
    {"0": "I", "1": "E"},
    {"0": "N", "1": "S"},
    {"0": "F", "1": "T"},
    {"0": "P", "1": "J"},
    ]

    result = []
    for num in combined:
        s = ""
        for i in range(len(num)):
            s += type_list[i][num[i]]
        result.append(s)
        
    return result


In [7]:
def predict(path_to_csv):

    df = pd.read_csv(path_to_csv)

    X, y = prep_data(df)

    # loading the 4 models
    EorI_model = load("clf_is_Extrovert.joblib")
    SorN_model = load( "clf_is_Sensing.joblib")
    TorF_model = load("clf_is_Thinking.joblib")
    JorP_model = load("clf_is_Judging.joblib")

    # predicting
    EorI_pred = EorI_model.predict(X)
    print(
        "Extrovert vs Introvert Accuracy: ",
        accuracy_score(y["is_Extrovert"], EorI_pred),
    )
    print("y_true", y["is_Extrovert"].values)
    print("preds", EorI_pred)

    SorN_pred = SorN_model.predict(X)
    print(
        "\nSensing vs Intuition Accuracy: ", accuracy_score(y["is_Sensing"], SorN_pred)
    )
    print("y_true", y["is_Sensing"].values)
    print("preds", SorN_pred)

    TorF_pred = TorF_model.predict(X)
    print(
        "\nThinking vs Feeling Accuracy: ", accuracy_score(y["is_Thinking"], TorF_pred)
    )
    print("y_true", y["is_Thinking"].values)
    print("preds", TorF_pred)

    JorP_pred = JorP_model.predict(X)
    print(
        "\nJudging vs Perceiving Accuracy: ", accuracy_score(y["is_Judging"], JorP_pred)
    )
    print("y_true", y["is_Judging"].values)
    print("preds", JorP_pred)

    # combining the predictions from the 4 models
    result = combine_classes(EorI_pred, SorN_pred, TorF_pred, JorP_pred)

    return result

In [8]:
if __name__ == "__main__":
    
    predictions = predict("df_holdout.csv")
    y_truth = pd.read_csv("df_holdout.csv")["type"].values


Total Preprocessing Time: 30.219797611236572 seconds

Extrovert vs Introvert Accuracy:  0.6206896551724138
y_true [0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0
 0 1 1 0 1 0 0 0 0 0 0 0 0]
preds [1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 1 0 0 0
 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 0
 1 1 1 0 1 0 0 0 1 1 0 0 0]

Sensing vs Intuition Accuracy:  0.6551724137931034
y_true [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 1 0 1 0 0 0 1]
preds [0 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 1 0 0 1 0 0 0 1 1 0 1]

Thinking vs Feeling Accuracy:  0.7931034482758621
y_true [0 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 1 

In [9]:
df["result"] = predictions
df

,type,posts,result
0,INFJ,'I have never seen so many poorly used memes.....,ENFP
1,INFJ,'Wow! You are obviously her muse... Be flatter...,ENFP
2,ENFP,'Are you shitting me? He's so type 7 its not e...,ESTP
3,ENTP,"'Oh man, this is serious. Good luck with her! ...",INTP
4,INFP,'Haha. Thank you! You guys have been so nice. ...,ENFP
...,...,...,...
82,ISTP,"'Hey, so incidentally, that's the exact same t...",ENTJ
83,INTJ,'I would answer your questions but I don't kno...,ESTJ
84,INFJ,"'I am the same. What I do, is send them occas...",ISFJ
85,INTP,'Important that i'm attracted to her at least ...,INFP


In [10]:
mbti = [
    "INFP",
    "INFJ",
    "INTP",
    "INTJ",
    "ENTP",
    "enfp",
    "ISTP",
    "ISFP",
    "ENTJ",
    "ISTJ",
    "ENFJ",
    "ISFJ",
    "ESTP",
    "ESFP",
    "ESFJ",
    "ESTJ",
]
tags_dict = {
    "ADJ_avg": ["JJ", "JJR", "JJS"],
    "ADP_avg": ["EX", "TO"],
    "ADV_avg": ["RB", "RBR", "RBS", "WRB"],
    "CONJ_avg": ["CC", "IN"],
    "DET_avg": ["DT", "PDT", "WDT"],
    "NOUN_avg": ["NN", "NNS", "NNP", "NNPS"],
    "NUM_avg": ["CD"],
    "PRT_avg": ["RP"],
    "PRON_avg": ["PRP", "PRP$", "WP", "WP$"],
    "VERB_avg": ["MD", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"],
    ".": ["#", "$", "''", "(", ")", ",", ".", ":"],
    "X": ["FW", "LS", "UH"],
}
features = [
    "clean_posts",
    "compound_sentiment",
    "ADJ_avg",
    "ADP_avg",
    "ADV_avg",
    "CONJ_avg",
    "DET_avg",
    "NOUN_avg",
    "NUM_avg",
    "PRT_avg",
    "PRON_avg",
    "VERB_avg",
    "qm",
    "em",
    "colons",
    "emojis",
    "word_count",
    "unique_words",
    "upper",
    "link_count",
    "ellipses",
    "img_count",
]


def unique_words(s):
    unique = set(s.split(" "))
    return len(unique)


def emojis(post):
    # does not include emojis made purely from symbols, only :word:
    emoji_count = 0
    words = post.split()
    for e in words:
        if "http" not in e:
            if e.count(":") == 2:
                emoji_count += 1
    return emoji_count


def colons(post):
    # Includes colons used in emojis
    colon_count = 0
    words = post.split()
    for e in words:
        if "http" not in e:
            colon_count += e.count(":")
    return colon_count


def lemmitize(s):
    lemmatizer = WordNetLemmatizer()
    new_s = ""
    for word in s.split(" "):
        lemmatizer.lemmatize(word)
        if word not in stopwords.words("english"):
            new_s += word + " "
    return new_s[:-1]


def clean(s):
    # remove urls
    s = re.sub(re.compile(r"https?:\/\/(www)?.?([A-Za-z_0-9-]+).*"), "", s)
    # remove emails
    s = re.sub(re.compile(r"\S+@\S+"), "", s)
    # remove punctuation
    s = re.sub(re.compile(r"[^a-z\s]"), "", s)
    # Make everything lowercase
    s = s.lower()
    # remove all personality types
    for type_word in mbti:
        s = s.replace(type_word.lower(), "")
    return s


def prep_counts(s):
    clean_s = clean(s)
    d = {
        "clean_posts": lemmitize(clean_s),
        "link_count": s.count("http"),
        "youtube": s.count("youtube") + s.count("youtu.be"),
        "img_count": len(re.findall(r"(\.jpg)|(\.jpeg)|(\.gif)|(\.png)", s)),
        "upper": len([x for x in s.split() if x.isupper()]),
        "char_count": len(s),
        "word_count": clean_s.count(" ") + 1,
        "qm": s.count("?"),
        "em": s.count("!"),
        "colons": colons(s),
        "emojis": emojis(s),
        "unique_words": unique_words(clean_s),
        "ellipses": len(re.findall(r"\.\.\.\ ", s)),
    }
    return clean_s, d


def prep_sentiment(s):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(s)
    d = {
        "compound_sentiment": score["compound"],
        "pos_sentiment": score["pos"],
        "neg_sentiment": score["neg"],
        "neu_sentiment": score["neu"],
    }
    return d


def tag_pos(s):
    tagged_words = nltk.pos_tag(word_tokenize(s))
    d = dict.fromkeys(tags_dict, 0)
    for tup in tagged_words:
        tag = tup[1]
        for key, val in tags_dict.items():
            if tag in val:
                tag = key
        d[tag] += 1
    return d


def prep_data(s):
    clean_s, d = prep_counts(s)
    d.update(prep_sentiment(lemmitize(clean_s)))
    d.update(tag_pos(clean_s))
    return pd.DataFrame([d])[features]


if __name__ == "__main__":
    t = time.time()
    string = "That somehow managed to be record short yet answer almost all the questions we would've asked, haha! Hi Deb! Welcome to Hou Tian; nice to meet you! I'm Jhenne, one of the mods here-- which means I gotta give you the modly speech :] Make sure to check out the Mandatory Reading up top! Our constantly updated Library is also a great resource, though it isn't mandatory reading-- we like to tell members to 'read as you need', rather than marathon read it all at once. One of the most helpful threads is the Gameplay So Far thread, which breaks down what all has gone down on the boards. (Now, the summary for January isn't tossed up yet, but we'd be happy to break down what you missed if you'd like.) I see that you're interested in Mai! That means both the Trying for a Canon Character page, and the Canon Character Rules and Consequences post will be helpful to check out. If you're ever considering an original character, we have our player-made adoptables list, and our factions, comprised of the Jade Shark/Bending Opposition, Original People of the Flame, and The Bending Crime Syndicates. As far as characters go, in the past tense I play Srai, a Jade Shark [s]that is very very dusty. In the Korraverse I play a reporter named Chihiro, and an ex-taxi dancer/wannabe actress named Naoki, and a Republic City University student named Haruna. I think that's it! If you have any questions, don't hesitate to ask a mod, or drop it right here in this thread so we can get back to you! Again, welcome! #CONFETTI"
    print(string)
    print(prep_data(string))
    print(f"Preprocessing Time: {time.time() - t} seconds")

That somehow managed to be record short yet answer almost all the questions we would've asked, haha! Hi Deb! Welcome to Hou Tian; nice to meet you! I'm Jhenne, one of the mods here-- which means I gotta give you the modly speech :] Make sure to check out the Mandatory Reading up top! Our constantly updated Library is also a great resource, though it isn't mandatory reading-- we like to tell members to 'read as you need', rather than marathon read it all at once. One of the most helpful threads is the Gameplay So Far thread, which breaks down what all has gone down on the boards. (Now, the summary for January isn't tossed up yet, but we'd be happy to break down what you missed if you'd like.) I see that you're interested in Mai! That means both the Trying for a Canon Character page, and the Canon Character Rules and Consequences post will be helpful to check out. If you're ever considering an original character, we have our player-made adoptables list, and our factions, comprised of the

In [11]:
def predict(s):
    return len(s.split(" "))


def predict_e(s):
    X = prep_data(s)

    EorI_model = load("clf_is_Extrovert.joblib")
    SorN_model = load("clf_is_Sensing.joblib")
    TorF_model = load("clf_is_Thinking.joblib")
    JorP_model = load("clf_is_Judging.joblib")

    EorI_pred = EorI_model.predict(X)[0]
    SorN_pred = SorN_model.predict(X)[0]
    TorF_pred = TorF_model.predict(X)[0]
    JorP_pred = JorP_model.predict(X)[0]

    # Mapping binary predictions to MBTI letters
    EorI_type = "E" if EorI_pred == 1 else "I"
    SorN_type = "S" if SorN_pred == 1 else "N"
    TorF_type = "T" if TorF_pred == 1 else "F"
    JorP_type = "J" if JorP_pred == 1 else "P"

    mbti_type = EorI_type + SorN_type + TorF_type + JorP_type
    print("Predicted personality type:", mbti_type)

    return mbti_type



if __name__ == "__main__":
    t = time.time()
    # sample test string. Type ISTP.
    string = "I plugged the data into tableau to see how the different features or how various mathematical formulas relate to the Weight. Once I had a few that didn’t have a wide distribution, I just started trying different models, even ones we hadn’t gone over yet. There are a LOT of regression models. I do not like this try everything method, it’s inefficient and illogical."
    print(predict_e(string))
    print(f"Preprocessing Time: {(time.time() - t):.2f} seconds")

Predicted personality type: INTP
INTP
Preprocessing Time: 0.84 seconds


In [12]:
def predict_e(s):
    X = prep_data(s)

    EorI_model = load("clf_is_Extrovert.joblib")
    SorN_model = load("clf_is_Sensing.joblib")
    TorF_model = load("clf_is_Thinking.joblib")
    JorP_model = load("clf_is_Judging.joblib")

    EorI_pred = EorI_model.predict(X)[0]
    SorN_pred = SorN_model.predict(X)[0]
    TorF_pred = TorF_model.predict(X)[0]
    JorP_pred = JorP_model.predict(X)[0]

    # Mapping binary predictions to MBTI letters
    EorI_type = "E" if EorI_pred == 1 else "I"
    SorN_type = "S" if SorN_pred == 1 else "N"
    TorF_type = "T" if TorF_pred == 1 else "F"
    JorP_type = "J" if JorP_pred == 1 else "P"

    mbti_type = EorI_type + SorN_type + TorF_type + JorP_type
    print("Predicted personality type:", mbti_type)

    return mbti_type

if __name__ == "__main__":
    t = time.time()
    string = "I plugged the data into tableau to see how the different features or how various mathematical formulas relate to the Weight. Once I had a few that didn’t have a wide distribution, I just started trying different models, even ones we hadn’t gone over yet. There are a LOT of regression models. I do not like this try everything method, it’s inefficient and illogical."
    print(predict_e(string))
    print(f"Preprocessing Time: {(time.time() - t):.2f} seconds")

Predicted personality type: INTP
INTP
Preprocessing Time: 1.17 seconds


In [15]:
if __name__ == "__main__":
    t = time.time()
    #string = "Hi everyone! I'm feeling energized today and excited to connect with all of you. I love being around people and making meaningful connections. Empathy and understanding are important values to me, and I always strive to support and uplift others. Let's work together to create positive change and make a difference in the world!"
    #string = "Why’s she here?Ah yes but what do I use for the rod. Those two cunts aren’t coming out.Forget what happened. Focus on the journey now.Everything’s alright with you dw about that , it’s okay to get mad. There’s no reason to get triggered like this Control your anger.You’re fluctuating way too much these days. Let’s just not jump in too much tho, can’t be perfect right? I get that classes are empty but there’s no point in shifting all 120 guys over there. idk not much work no? So there’s no point in finishing it immediately no?"
    #string = "For the current 6th ( going to 7th) a company is ready to give many internship in  AI applied to  cyber security provided the students knoNote that 'python' is not equal to ML.  ML= Mathsw basic maths.Institute project is about applications of LLM. Need lots of data preparation job. Those who are really hardworking and show real attitude, we will recommend to companies where Amrita Alumnus are leading the AI Software  development.|||SVD,  Pseudoinverse and ADMM are the three pillars of AI and Datascience. We expect all to master this and can be asked  in any viva ( in any subject). No 'A' grade without proper knowledge of these three topics.There will be a workshop on RDkit for bio and chemo_informatics. Knowledge in many domains (like above+NLP+Vision+ +finance+robotics etc) is needed to get a job."
    #string = "I've been reflecting a lot on how we can make the world a kinder place, and I think small acts of compassion can create a ripple effect of positive change. Sometimes, it's the little things that make the biggest difference in someone's day. I love finding unique ways to express my creativity, whether it's through writing, art, or simply daydreaming about new ideas. It's important for me to stay true to my values and be authentic in everything I do. I believe that if we all listened to our hearts more, the world would be a more harmonious place"
    string = "I guess yeh life long chalega you know||| Theres this one fine looking dude i saw here tho||| Lets go hyderabdi hunting ||| Dude that thing was on 31 next day after i came there wasnt time to ||| Ok question is just where do we drink ||| Like meko akele toh obviosly nahi pina h ||| i dont like cheese ||| I dont talk alot."
    #string =  "Dear Dr. Ramesh Sivan Pillai from University of Wyoming USA is giving a 1 hour ta
    print(predict_e(string))
    print(f"Processing Time: {(time.time() - t):.2f} seconds")

Predicted personality type: INFP
INFP
Processing Time: 0.74 seconds
